# Ribonanza 2

A second approach to the [Stanford Ribonanza problem](https://www.kaggle.com/competitions/stanford-ribonanza-rna-folding/) that builds off the first approach.

Major differences:
- use of different model architecture
- use of only filtered data (data in which SN_filter == 1)

Currently, this scores a 0.26166 MAE

 ![Alt text](image.png)

## Setup

### Filesystem Setup

Your project directory should look like this:

- `(project directory)`
    - `ribonanza2.ipynb`
    - `train_data.csv`

`train_data.csv` is the only file necessary, and it can be downloaded from the kaggle competition linked in the description.

### Code Setup

In [1]:
# imports
import tensorflow as tf
import keras
import keras.layers as layers
import keras.optimizers as optimizers
import pandas
import numpy as np
from tqdm import tqdm
import seaborn
import os

2023-10-02 23:16:19.262193: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-02 23:16:20.000560: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# constants

# according to kaggle, this is the maximum # of reactivites to be used
NUM_REACTIVITIES = 457

# there are 4 different bases (AUCG)
NUM_BASES = 4

## Data Preprocessing

### Filter Data

In [3]:
def filter_data(out: str, key: str, value: str, file_name: str, force: bool):
    """
    Filters a file to only take datapoints
    whose values of `key` are `value`.

    Parameters:
        - out: str - the name of the file that will store the filtered datapoints
        - key: str - the name of the key to look at
        - value: str - the value that the key should have
        - file_name: str - the name of the file that contains all the datapoints.
        - force: bool - whether or not to force re-processing of the data (if False and `out` already exists, no work will be done)
    """
    if os.path.exists(out) and not force:
        print("File already exists, not doing any work")
        return

    count = 0

    # count how many lines we have in total
    with open(file_name) as file:
        line = file.readline()  # ignore the header
        line = (
            file.readline()
        )  # take the first line since we increment count in the loop
        while line != "":
            count += 1
            line = file.readline()

    # use that knowledge for a progress bar
    with open(file_name, "r") as file, open(out, "w") as outfile:
        # write the header
        header = file.readline()
        outfile.write(header)

        # get what index the SN_filter is
        SN_idx = header.split(",").index(key)

        # only take the approved filtered lines
        for _ in tqdm(range(count)):
            line = file.readline()
            temp = line.split(",")
            if temp[SN_idx] == value:
                outfile.write(line)


def filter_train_data(force: bool = False):
    """
    Filters the immense train_data.csv to only take datapoints
    whose SN_filter (Signal to Noise filter) is 1. In other words,
    we only take good reads. These filtered datapoints are then
    written to the file provided

    Parameters:
        - force: bool - whether or not to force re-processing of the data (if False and `out` already exists, no work will be done)
    """
    filter_data("train_data_filtered.csv", "SN_filter", "1", "train_data.csv", force)


def filter_2A3(force: bool = False):
    """
    Only take the 2A3 points

    Parameters:
        - force: bool - whether or not to force re-processing of the data (if False and `out` already exists, no work will be done)
    """
    filter_data(
        "train_data_2a3.csv",
        "experiment_type",
        "2A3_MaP",
        "train_data_filtered.csv",
        force,
    )


def filter_DMS(force: bool = False):
    """
    Only take the DMS points

    Parameters:
        - force: bool - whether or not to force re-processing of the data (if False and `out` already exists, no work will be done)
    """
    filter_data(
        "train_data_dms.csv",
        "experiment_type",
        "DMS_MaP",
        "train_data_filtered.csv",
        force,
    )

In [4]:
# filter our data
filter_train_data()

File already exists, not doing any work


In [5]:
# take the 2a3 points
filter_2A3()

File already exists, not doing any work


In [6]:
# take the dms points
filter_DMS()

File already exists, not doing any work


### Convert Data to Inputs and Outputs

In [7]:
# encode inputs as
# A : [1, 0, 0, 0]
# U : [0, 1, 0, 0]
# C : [0, 0, 1, 0]
# G : [0, 0, 0, 1]
base_map = {
    "A": np.array([1, 0, 0, 0]),
    "U": np.array([0, 1, 0, 0]),
    "C": np.array([0, 0, 1, 0]),
    "G": np.array([0, 0, 0, 1]),
}

In [8]:
def preprocess_csv(out: str, file_name: str, force: bool = False):
    """
    Preprocess the csv and save the preprocessed data as a .npz file

    Parameters:
        - out: str - the name of the file to save the arrays to
        - file_name: str - the name of the input csv file
        - force: bool - whether or not to force re-processing of the data (if False and `out` already exists, no work will be done).
                Defaults to `False`
    """
    if os.path.exists(out) and not force:
        print("File already exists, not doing any work")
        return

    df = pandas.read_csv(file_name)

    inputs = np.zeros((len(df), NUM_REACTIVITIES, NUM_BASES))
    outputs = np.zeros((len(df), NUM_REACTIVITIES))
    output_masks = np.ones((len(df), NUM_REACTIVITIES), dtype=np.bool_)

    for index in tqdm(range(len(df))):
        row = df.iloc[index]

        # get the sequence
        seq_len = len(row["sequence"])

        # map the base to its one-hot encoding
        inputs[index, :seq_len] = np.array(
            list(map(lambda letter: base_map[letter], row["sequence"]))
        )

        # get all the reactivities and whether or not they are nan
        reactivities = np.array(
            list(
                map(
                    lambda seq_idx: row["reactivity_" + str(seq_idx + 1).rjust(4, "0")],
                    range(seq_len),
                )
            )
        )
        nan_locats = np.isnan(reactivities)

        # where it is nan, store True, else false
        output_masks[index, :seq_len] = nan_locats

        # where it is not nan, store the reactiviy, else 0
        outputs[index, :seq_len] = np.where(nan_locats == False, reactivities, 0.0)

    # save the outputs
    np.savez_compressed(out, inputs=inputs, outputs=outputs, output_masks=output_masks)

In [9]:
preprocess_csv("train_data_2a3_preprocessed.npz", "train_data_2a3.csv")

File already exists, not doing any work


In [10]:
preprocess_csv("train_data_dms_preprocessed.npz", "train_data_dms.csv")

File already exists, not doing any work


### Load the desired dataset

In [11]:
desired_dataset = "2a3"  # either "2a3" or "dms"


In [12]:
# load the npz file
npz_file = np.load(f"train_data_{desired_dataset}_preprocessed.npz")

# stored inputs, outputs, and output_masks
inputs, outputs, output_masks = (
    npz_file["inputs"],
    npz_file["outputs"],
    npz_file["output_masks"],
)

# close the npz file
npz_file.close()

In [13]:
# convert to usable weights:
# if it is meant to be masked, it should be worth 0, else it should be worth 1
output_masks = np.where(output_masks, 0, 1)
output_masks.shape

(210992, 457)

### Visualize

In [14]:
visualize = False

In [15]:
if visualize:
    visualized_items = []
    for i in tqdm(range(len(outputs))):
        for x in range(NUM_REACTIVITIES):
            if not output_masks[i, x]:
                visualized_items.append(outputs[i, x])
    visualized_items = np.array(visualized_items)
else:
    print("Not visualizing. Set `visualize` to `True` to visualize data")

Not visualizing. Set `visualize` to `True` to visualize data


In [16]:
if visualize:
    seaborn.histplot(visualized_items, stat="count", binwidth=0.1)
else:
    print("Not visualizing. Set `visualize` to `True` to visualize data")

Not visualizing. Set `visualize` to `True` to visualize data


## Model

In [17]:
def make_baseline_model():
    inputs = layers.Input((NUM_REACTIVITIES, NUM_BASES))
    
    x = layers.Conv1D(4, 16, 1, padding='same', activation='relu')(inputs)
    x = layers.Conv1D(4, 16, 1, padding='same', activation='relu')(x)
    x = layers.Conv1D(4, 16, 2, padding='same', activation='relu')(x)
    x = layers.Flatten()(x)
    x = layers.Dense(1024, activation="relu", activity_regularizer="l2")(x)

    x = layers.Dense(NUM_REACTIVITIES)(x)

    return keras.Model(inputs=inputs, outputs=x)


In [18]:
model = make_baseline_model()
model.summary()

2023-10-02 23:16:31.358617: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-02 23:16:31.387324: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-02 23:16:31.387564: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 457, 4)]          0         
                                                                 
 conv1d (Conv1D)             (None, 457, 4)            260       
                                                                 
 conv1d_1 (Conv1D)           (None, 457, 4)            260       
                                                                 
 conv1d_2 (Conv1D)           (None, 229, 4)            260       
                                                                 
 flatten (Flatten)           (None, 916)               0         
                                                                 
 dense (Dense)               (None, 1024)              939008    
                                                                 
 dense_1 (Dense)             (None, 457)               468425

In [19]:
model.compile(optimizer = optimizers.Adam(1e-4), loss=tf.keras.losses.MeanAbsoluteError())

## Train

In [20]:
@tf.function
def train_batch(m:keras.Model, inps:tf.Tensor, outs:tf.Tensor, masks:tf.Tensor):
    with tf.GradientTape() as tape:
        preds = tf.expand_dims(m(inps, training=True), axis=-1)
        outs = tf.expand_dims(outs, axis=-1)

        loss = 0.0
        for b in range(inps.shape[0]):
            loss += m.loss(preds[b], outs[b], masks[b])
        
        # turn it into mean
        loss /= inps.shape[0]
        
        # add the regularization losses
        loss += model.losses
        # calculate gradients
        grads = tape.gradient(loss, model.trainable_variables)
        
    # apply grads
    model.optimizer.apply_gradients(zip(grads, model.trainable_variables))

    # return total loss, mae loss
    return loss, loss - model.losses

@tf.function
def noupdate_batch(m:keras.Model, inps:tf.Tensor, outs:tf.Tensor, masks:tf.Tensor):
    preds = tf.expand_dims(m(inps, training=False), axis=-1)
    outs = tf.expand_dims(outs, axis=-1)
    
    loss = 0.0
    for b in range(inps.shape[0]):
        loss += m.loss(preds[b], outs[b], masks[b])
    
    # turn it into mean
    loss /= inps.shape[0]
    
    # add the regularization losses
    loss += model.losses
        
    # return total loss, mae loss
    return loss, loss - model.losses

def masked_train(m: keras.Model, x: np.ndarray, y: np.ndarray, masks: np.ndarray, batch_size: int = 32, epochs: int = 1, validation_split: float = 0.1):
    # shuffle
    # shuffled_idxs = np.arange(x.shape[0])
    # np.random.shuffle(shuffled_idxs)
    # x = x[shuffled_idxs]
    # y = y[shuffled_idxs]
    # masks = masks[shuffled_idxs]
    
    # generate validation
    validation_size = int(x.shape[0] * validation_split)
    x_val = x[:validation_size]
    y_val = y[:validation_size]
    masks_val= masks[:validation_size]
    x = x[validation_size:]
    y = y[validation_size:]
    masks = masks[validation_size:]

    # calculate number of batches to do
    num_batches = x.shape[0] // batch_size
    if x.shape[0] % batch_size != 0:
        num_batches += 1

    num_validation_batches = x_val.shape[0] // batch_size
    if x_val.shape[0] % batch_size != 0:
        num_validation_batches += 1

    for epoch in range(1, epochs+1):
        print(f"Epoch {epoch}")
        epoch_loss = 0.0
        epoch_mae = 0.0

        for batch in range(num_batches):
            num_items = min(batch_size, x.shape[0] - batch*batch_size)

            inps = tf.constant(x[batch*batch_size:batch*batch_size+num_items])
            outs = tf.constant(y[batch*batch_size:batch*batch_size+num_items])
            masks_ = tf.constant(masks[batch*batch_size:batch*batch_size+num_items])

            loss, mae_loss = train_batch(m, inps, outs, masks_)

            epoch_loss += loss
            epoch_mae += mae_loss

            # log
            print(f"Batch {batch+1}/{num_batches}\t- loss: {loss[0].numpy():.5f}\t- mae loss: {mae_loss[0].numpy():.5f}", end="\r")
        epoch_loss /= num_batches
        epoch_mae /= num_batches

        # do validation
        val_loss = 0.0
        val_mae = 0.0
        for batch in range(num_validation_batches):
            num_items = min(batch_size, x.shape[0] - batch*batch_size)

            inps = tf.constant(x_val[batch*batch_size:batch*batch_size+num_items])
            outs = tf.constant(y_val[batch*batch_size:batch*batch_size+num_items])
            masks_ = tf.constant(masks_val[batch*batch_size:batch*batch_size+num_items])

            loss, mae_loss = noupdate_batch(m, inps, outs, masks_)

            val_loss += loss
            val_mae += mae_loss
        val_loss /= num_validation_batches
        val_mae /= num_validation_batches

        # shuffle
        # shuffled_idxs = np.arange(x.shape[0])
        # np.random.shuffle(shuffled_idxs)
        # x = x[shuffled_idxs]
        # y = y[shuffled_idxs]
        # masks = masks[shuffled_idxs]
        
        print()
        print(f"Epoch loss: {epoch_loss[0]:.5f}\tEpoch MAE: {epoch_mae[0]:.5f}\tVal loss: {val_loss[0]:.5f}\tVal MAE: {val_mae[0]:.5f}")

In [21]:
masked_train(model, inputs, outputs, output_masks, epochs=5, batch_size=128)

Epoch 1


2023-10-02 23:16:41.533805: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-10-02 23:16:42.377644: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


Batch 1484/1484	- loss: 0.08924	- mae loss: 0.08902
Epoch loss: 0.08359	Epoch MAE: 0.08221	Val loss: 0.08229	Val MAE: 0.08194
Epoch 2
Batch 1484/1484	- loss: 0.08722	- mae loss: 0.08690
Epoch loss: 0.08191	Epoch MAE: 0.08142	Val loss: 0.08149	Val MAE: 0.08107
Epoch 3
Batch 1484/1484	- loss: 0.08243	- mae loss: 0.08147
Epoch loss: 0.08162	Epoch MAE: 0.08122	Val loss: 0.08138	Val MAE: 0.08092
Epoch 4
Batch 1484/1484	- loss: 0.07300	- mae loss: 0.06994
Epoch loss: 0.08144	Epoch MAE: 0.08109	Val loss: 0.08126	Val MAE: 0.08064
Epoch 5
Batch 1484/1484	- loss: 0.06245	- mae loss: 0.05555
Epoch loss: 0.08120	Epoch MAE: 0.08081	Val loss: 0.08105	Val MAE: 0.08044


## Save

In [22]:
model.save(f"{desired_dataset}_model")

INFO:tensorflow:Assets written to: 2a3_model/assets


INFO:tensorflow:Assets written to: 2a3_model/assets


## Process Outputs

In [23]:
valid = False
if os.path.exists("2a3_model") and os.path.exists("dms_model"):
    valid = True
    model_2a3 = keras.models.load_model("2a3_model")
    model_dms = keras.models.load_model("dms_model")    

In [24]:
@tf.function
def call_model(model_2a3, model_dms, inputs):
    return model_2a3(inputs, training=False), model_dms(inputs, training=False)

def pipeline(model_2a3:keras.Model, model_dms:keras.Model, input_csv:str, out:str, batch_size:int):
    count = 0

    # count how many lines we have in total
    with open(input_csv) as file:
        line = file.readline()  # ignore the header
        # take the first line since we increment count in the loop
        line = file.readline()
        while line != "":
            count += 1
            line = file.readline()

    # use that knowledge for a progress bar
    with open(input_csv, "r") as file, open(out, "w") as outfile:
        # write the header
        outfile.write("id,reactivity_DMS_MaP,reactivity_2A3_MaP\n")

        # get what index the things we need are
        header = file.readline()
        split_header = header.split(",")
        min_idx = split_header.index("id_min")
        max_idx = split_header.index("id_max")
        sequence_idx = split_header.index("sequence")

        # only take the approved filtered lines
        num_batches = count // batch_size
        if count % batch_size != 0:
            num_batches += 1
        for batch in tqdm(range(num_batches)):
            num_items = min(batch_size, count - batch*batch_size)

            inputs = np.zeros((num_items, NUM_REACTIVITIES, NUM_BASES))
            min_seq_idxs = []
            sequence_lengths = []
            
            for i in range(num_items):
                line = file.readline()
                temp = line.split(",")
                sequence = temp[sequence_idx]
                max_seq_idx = int(temp[max_idx])
                min_seq_idx = int(temp[min_idx])
                assert len(sequence) + min_seq_idx -1 == max_seq_idx

                inputs[i, :len(sequence)] = np.array(list(map(lambda letter: base_map[letter], sequence)))
                min_seq_idxs.append(min_seq_idx)
                sequence_lengths.append(len(sequence))

            # run it through the associated model
            probs_2a3, probs_dms = call_model(model_2a3, model_dms, inputs)
            probs_dms = probs_dms.numpy()
            probs_2a3 = probs_2a3.numpy()

            for i in range(num_items):
                for seq_idx in range(min_seq_idxs[i], min_seq_idxs[i] + sequence_lengths[i]):
                    outfile.write(f"{seq_idx},{probs_dms[i, seq_idx - min_seq_idxs[i]]:.3f},{probs_2a3[i, seq_idx - min_seq_idxs[i]]:.3f}\n")

In [25]:
if valid:
    pipeline(model_2a3, model_dms, "test_sequences.csv", "submission.csv", batch_size=256)

100%|██████████| 5250/5250 [07:22<00:00, 11.86it/s]


In [ ]:
# zip our submission into an easily-uploadable zip file
os.system("zip submission.csv.zip submission.csv")